<div align=right>
Winter 2025<br>
Nardin<br>
HW3<br>
Yolanda Pan
</div>


<font color='darkblue'> <h2 align=center> Homework 3: Sqlite3</h2> </font>

<font color='darkblue'> <h3 align=left> Instructions </h3> </font>

Please complete the two tasks described below. Add as many code blocks as you'd like. 

Submit on Gradescope your Jupyter notebook with the homework completed and the output of your code visible. Ensure the file is not too large for Gradescope. Do not submit the data, we only need the notebook.

HWs must be completed individually and are graded on a Pass-Fail basis. Before submitting, see the Syllabus for homework policies (what qualifies as a "Pass", deadlines, regrading, etc.)

**At the end of the notebook (submissions without all of the following will automatically be graded as "Fail"):**
* List all resources you consulted (e.g., if you consulted specific websites, please add all links to them; if you used AI, explain for which task you used it and how). Do not submit code you do not fully understand or have not written yourself.
* Write a few sentences reflecting on what you learned from this homework.

<font color='darkblue'> <h3 align=left> Task 1</h3> </font>

In class, we worked with a political relational database of contributions and candidates. Review the key class materials. Then, write the following queries and fetch the results:

1. Load the required libraries. Generate the database (to do so you need the contributions.csv, candidate.csv, and the sql schema), open a connection to it, and create a cursor object.

In [1]:
# load the required libaries
import sqlite3
import pandas as pd

# generate the database
! sqlite3 fec.db < create-fec.sql

# connect with the db and create a cursor object
conn = sqlite3.connect("fec.db")
cur = conn.cursor()

2. Display the tables included in your database, and display the schema and attributes of all tables.

In [2]:
# display the tables included in the databse
c = "SELECT name FROM sqlite_master WHERE type = 'table'"
tabels = cur.execute(c).fetchall()
print(tabels)
print('\n')

# display the schema and attributes of all tables
for table in tabels:
    c = f"PRAGMA table_info({table[0]})"
    schema = cur.execute(c).fetchall()
    print(f'Schema for table {table[0]}')
    for row in schema:
        print(row)
    print('\n')


[('candidate',), ('contribution',)]


Schema for table candidate
(0, 'committee_id', 'varchar(10)', 0, None, 1)
(1, 'lastname', 'varchar(50)', 0, None, 0)
(2, 'firstname', 'varchar(50)', 0, None, 0)
(3, 'party', 'varchar(6)', 0, None, 0)
(4, 'city', 'varchar(20)', 0, None, 0)
(5, 'state', 'varchar(2)', 0, None, 0)
(6, 'zip', 'INTEGER', 0, None, 0)
(7, 'cand_id', 'varchar(10)', 0, None, 0)
(8, 'district', 'varchar(2)', 0, None, 0)


Schema for table contribution
(0, 'cand_id', 'varchar(10)', 0, None, 0)
(1, 'amount', 'INTEGER', 0, None, 0)
(2, 'city', 'varchar(20)', 0, None, 0)
(3, 'state', 'varchar(10)', 0, None, 0)
(4, 'zip', 'INTEGER', 0, None, 0)
(5, 'month', 'INTEGER', 0, None, 0)
(6, 'year', 'INTEGER', 0, None, 0)




3. From the candidate table, select only the column or attribute "state" (selecting columns is called a "projection operation" see in-class notebooks for details), and compute the number of candidates for each state. Display the results in descending order. 

In [8]:
q = '''
    SELECT state, COUNT(cand_id) AS cnt
    FROM contribution
    GROUP BY state
    ORDER BY cnt DESC
    '''

cur.execute(q).fetchall()

[('CA', 1428),
 ('VA', 1195),
 ('DC', 896),
 ('MO', 683),
 ('WA', 601),
 ('MD', 503),
 ('WI', 438),
 ('VT', 412),
 ('AK', 408),
 ('PA', 392),
 ('FL', 372),
 ('NY', 370),
 ('MI', 355),
 ('NJ', 281),
 ('TX', 231),
 ('MA', 147),
 ('IL', 125),
 ('AZ', 122),
 ('KS', 100),
 ('UT', 78),
 ('NV', 74),
 ('OH', 59),
 ('CT', 58),
 ('LA', 55),
 ('VI', 49),
 ('IN', 49),
 ('WV', 43),
 ('GA', 43),
 ('CO', 41),
 ('OK', 40),
 ('MN', 37),
 ('PR', 33),
 ('TN', 31),
 ('AL', 27),
 ('OR', 26),
 ('NC', 26),
 ('GU', 19),
 ('MS', 17),
 ('  ', 17),
 ('NH', 14),
 ('NM', 13),
 ('RI', 12),
 ('AR', 11),
 ('DE', 10),
 ('WY', 9),
 ('SC', 8),
 ('ME', 8),
 ('KY', 7),
 ('ID', 7),
 ('NE', 4),
 ('HI', 4),
 ('MT', 3),
 ('MP', 3),
 ('ND', 2),
 ('IA', 2)]

4. From the contribution table, calculate the total amount of contributions for each city. Display only the first 10 results in descending order and make the query case insensitive. Which city got the highest amount of total contributions? 

In [9]:
q = '''
    SELECT city, SUM(amount) AS total
    FROM contribution
    GROUP BY city COLLATE NOCASE
    ORDER BY total DESC
    LIMIT 10
    '''

cur.execute(q).fetchall()

[('Washington', 664430),
 ('Los Angeles', 267356),
 ('New York', 255295),
 ('Arlington', 213551),
 ('Alexandria', 194696),
 ('Seattle', 132550),
 ('Anchorage', 131401),
 ('Bethesda', 110402),
 ('Beverly Hills', 93100),
 ('Mc Lean', 88599)]

5. Calculate the total contributions received by each candidate (display their first and last name) in Illinois in 2010, ordered in descending order. Your query should link candidate with their contributions (e.g.,  join the two tables) and should take the candidate data from the candidate table, and the contributions, state, and year data from the contribution table.

In [21]:
q = '''
    SELECT firstname, lastname, SUM(amount) AS total
    FROM candidate
    LEFT JOIN contribution
    ON candidate.cand_id = contribution.cand_id
    WHERE contribution.state = 'IL' COLLATE NOCASE
    AND year = 2010
    GROUP BY candidate.cand_id
    ORDER BY total DESC
    '''

cur.execute(q).fetchall()

[(' PATRICK', 'LEAHY', 19600),
 (' ORRIN G', 'HATCH', 14099),
 (' IKE', 'SKELTON', 6300),
 (' NORMAN D', 'DICKS', 5000),
 (' DAVID R.', 'OBEY', 4250),
 (' CHRISTOPHER H.', 'SMITH', 3500),
 (' JOHN D. MR.', 'DINGELL', 3500),
 (' GEORGE', 'MILLER', 1500),
 (' NICK JOE II', 'RAHALL', 1000),
 (' DAVID', 'DREIER', 250)]

6. Rewrite the previous query (question 5) using parameters so it displays results for three states of your choice and two years (2009 and 2010).

In [23]:
q = '''
    SELECT firstname, lastname, SUM(amount) AS total, year
    FROM candidate
    LEFT JOIN contribution
    ON candidate.cand_id = contribution.cand_id
    WHERE contribution.state = ? COLLATE NOCASE
    AND year IN (?, ?)
    GROUP BY candidate.cand_id
    ORDER BY total DESC
    '''

# define parameter
state = ['IL']
year = [2009, 2010]

cur.execute(q, state + year).fetchall()

[(' DAVID R.', 'OBEY', 37350, 2009),
 (' PATRICK', 'LEAHY', 29900, 2009),
 (' ORRIN G', 'HATCH', 20698, 2009),
 (' IKE', 'SKELTON', 13600, 2009),
 (' JOHN P MR.', 'MURTHA', 7200, 2009),
 (' NORMAN D', 'DICKS', 5250, 2009),
 (' JOHN D. MR.', 'DINGELL', 5000, 2009),
 (' CHRISTOPHER H.', 'SMITH', 4400, 2009),
 (' GEORGE', 'MILLER', 3750, 2009),
 (' HENRY A.', 'WAXMAN', 2500, 2009),
 (' NICK JOE II', 'RAHALL', 2250, 2009),
 (' DAVID', 'DREIER', 1850, 2009)]

In [15]:
# alternative with placeholder
placeholders = ', '.join('?' * len(year))

q = '''
    SELECT firstname, lastname, SUM(amount) AS total
    FROM candidate
    LEFT JOIN contribution
    ON candidate.cand_id = contribution.cand_id
    WHERE contribution.state = ? COLLATE NOCASE
    AND year IN ({})
    GROUP BY candidate.cand_id
    ORDER BY total DESC
    '''.format(placeholders)

cur.execute(q, state + year).fetchall()

[(' PATRICK', 'LEAHY', 19600),
 (' ORRIN G', 'HATCH', 14099),
 (' IKE', 'SKELTON', 6300),
 (' NORMAN D', 'DICKS', 5000),
 (' DAVID R.', 'OBEY', 4250),
 (' CHRISTOPHER H.', 'SMITH', 3500),
 (' JOHN D. MR.', 'DINGELL', 3500),
 (' GEORGE', 'MILLER', 1500),
 (' NICK JOE II', 'RAHALL', 1000),
 (' DAVID', 'DREIER', 250)]

7. Close the connection with this db.

In [16]:
conn.close()

<font color='darkblue'> <h3 align=left> Task 2</h3> </font>

For this homework, you will be using the same database of the second programming assigment (PA2). The db is called `course_information.sqlite3` and placed a copy of it in the same working directory as this notebook. Explore the db (using sql or pandas or both), then write the queries listed below and fetch the results. Submit a jupyter notebook that only prints the total number of results (e.g., the number 51 for query 1, not the actualy 51 query results).

1. Create a connection with the database `course_information.sqlite3` and create a cursor object.

In [8]:
import sqlite3
import pandas as pd

In [9]:
conn = sqlite3.connect("course_information.sqlite3")
cur = conn.cursor()

2. Display the tables included in your database, and display the schema and attributes of all tables.

In [10]:
# display the tables included in the databse
c = "SELECT name FROM sqlite_master WHERE type = 'table'"
tabels = cur.execute(c).fetchall()
print(pd.read_sql_query(c, conn))

print("\n")

# display the schema and attributes of all tables
for table in tabels:
    table = table[0]
    q = f"PRAGMA table_info({table})"
    print(f'Schema for table {table}')
    print(pd.read_sql_query(q, conn))
    print("\n")

               name
0           courses
1  meeting_patterns
2          sections
3               gps
4     catalog_index


Schema for table courses
   cid        name          type  notnull dflt_value  pk
0    0   course_id       INTEGER        0       None   0
1    1        dept    varchar(4)        0       None   0
2    2  course_num    varchar(5)        0       None   0
3    3       title  varchar(100)        0       None   0


Schema for table meeting_patterns
   cid                name         type  notnull dflt_value  pk
0    0  meeting_pattern_id      INTEGER        0       None   0
1    1                 day  varchar (5)        0       None   0
2    2          time_start      INTEGER        0       None   0
3    3            time_end      INTEGER        0       None   0


Schema for table sections
   cid                name        type  notnull dflt_value  pk
0    0          section_id     INTEGER        0       None   0
1    1           course_id     INTEGER        0       None

3. Find the titles of all courses with department code "CMSC" in the course table. Expected results: 51.

In [11]:
q = """
    SELECT title
    FROM courses
    WHERE dept = 'CMSC'
    """
# cmsc_courses = cur.execute(q).fetchall()
# print(len(cmsc_courses))
# print(cmsc_courses)

df = pd.read_sql_query(q, conn)
df

,title
0,Introduction to Computer Science I.
1,Computer Science with Applications II.
2,Implementation of Computer Languages II.
3,Networks.
4,Machine Learning.
5,Mobile Computing.
6,Introduction to Database Systems.
7,Software Construction.
8,Introduction to Formal Languages.
9,Implementation of Computer Languages I.


4. Find the department names, course numbers, and section numbers for courses being offered on MWF at 10:30am (represented as 1030). Expected results: 68.

In [12]:
q = """
    SELECT c.dept, c.course_num, s.section_num
    FROM courses c
    LEFT JOIN sections s
    ON c.course_id = s.course_id
    LEFT JOIN meeting_patterns mp
    ON s.meeting_pattern_id = mp.meeting_pattern_id
    WHERE mp.day = 'MWF'AND mp.time_start = 1030
    ORDER BY c.dept
    """ 

# result = cur.execute(q).fetchall()
# print(len(result))
# print(result)

df = pd.read_sql_query(q, conn)
df

,dept,course_num,section_num
0,AKKD,10102,01
1,ARAB,10102,02
2,BCSN,10203,01
3,BIOS,20187,BB
4,BIOS,23249,01
...,...,...,...
63,SPAN,10200,02
64,SPAN,20300,01
65,STAT,22000,01
66,TBTN,10200,01


5. Find the department names and course numbers for courses being offered in Ryerson (building code: "RY") on MWF between 10:30am and 3pm (represented as 1500). Expected results: 11 (or 9 without duplicates).

In [13]:
q = """
    SELECT c.dept, c.course_num
    FROM courses c
    LEFT JOIN sections s
    ON c.course_id = s.course_id
    LEFT JOIN meeting_patterns mp
    ON s.meeting_pattern_id = mp.meeting_pattern_id
    WHERE s.building_code = 'RY'
    AND mp.day = 'MWF'
    AND mp.time_start >= 1030 AND mp.time_end <= 1500
    ORDER BY c.dept
    """ 

# result = cur.execute(q).fetchall()
# print(len(result))
# print(result)

df = pd.read_sql_query(q, conn)
df

,dept,course_num
0,CMSC,15100
1,CMSC,27200
2,CMSC,23200
3,MATH,16200
4,MATH,19520
5,MATH,16200
6,MATH,16200
7,MATH,20400
8,MATH,20300
9,MATH,13200


In [14]:
q = """
    SELECT DISTINCT c.dept, c.course_num
    FROM courses c
    LEFT JOIN sections s
    ON c.course_id = s.course_id
    LEFT JOIN meeting_patterns mp
    ON s.meeting_pattern_id = mp.meeting_pattern_id
    WHERE s.building_code = 'RY'
    AND mp.day = 'MWF'
    AND mp.time_start >= 1030 AND mp.time_end <= 1500
    ORDER BY c.dept
    """ 

# result = cur.execute(q).fetchall()
# print(len(result))
# print(result)


df = pd.read_sql_query(q, conn)
df

,dept,course_num
0,CMSC,15100
1,CMSC,27200
2,CMSC,23200
3,MATH,16200
4,MATH,19520
5,MATH,20400
6,MATH,20300
7,MATH,13200
8,MATH,20000


6. Find the department names, course numbers, and course titles for courses being offered on MWF at 9:30am (represented as 930) that have the words "programming" or "abstraction" in their title/course description. Write the query using parameters. Expected results: 3 (or 2 without duplicates).

In [49]:
q = f"""
    SELECT DISTINCT c.dept, c.course_num, c.title
    FROM courses c
    JOIN catalog_index cat ON c.course_id = cat.course_id
    JOIN sections s ON c.course_id = s.course_id
    JOIN meeting_patterns mp 
    ON s.meeting_pattern_id = mp.meeting_pattern_id
    WHERE mp.day = ?
    AND mp.time_start = ?
    AND (cat.word IN ({', '.join(['?' for _ in word_lst])}) COLLATE NOCASE)
    """

day = 'MWF'
start_time = 930
word_lst = ['programming', 'abstraction']
params = [day, start_time] + word_lst
df = pd.read_sql_query(q, conn, params = params)
df

,dept,course_num,title
0,CMSC,12200,Computer Science with Applications II.
1,CMSC,15200,Introduction to Computer Science II.


7. Close the connection with this db.

In [50]:
conn.close()

<font color='darkblue'> <h3 align=left> Reflections and Resources </h3> </font>

1. all jupyter notebooks from class
2. I asked GPT the grammar of LIKE